# college chatbot model train 


In [1]:
import pandas as pd
from datasets import load_dataset
squad = load_dataset('squad')


Using the latest cached version of the dataset since squad couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'plain_text' at C:\Users\DeLL\.cache\huggingface\datasets\squad\plain_text\0.0.0\7b6d24c440a36b6815f21b70d25016731768db1f (last modified on Thu Sep 19 14:10:52 2024).


# import 

In [2]:
# Inspect the structure of the dataset
print(squad)

# View a sample question-answer pair
print(squad['train'][0])


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})
{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome

# to be used transformer build tokenizer

In [5]:
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')



c:\Users\DeLL\python\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# build suaq tokenizer to train nlp model

In [42]:
def add_token_positions(examples):
    tokenized_examples = tokenizer(
        examples["question"], examples["context"], 
        truncation=True, padding='max_length', max_length=512, return_offsets_mapping=True
    )
    
    # Initialize empty lists for start and end positions
    start_positions = []
    end_positions = []

    # Loop through each example
    for i, offsets in enumerate(tokenized_examples["offset_mapping"]):
        # Get the answer's start and end position in characters
        start_char = examples["answers"][i]['answer_start'][0]
        end_char = start_char + len(examples["answers"][i]['text'][0])

        # Initialize token start and end indices
        token_start_index = 0
        token_end_index = 0

        # Find the token index that corresponds to the character positions
        for j, (start, end) in enumerate(offsets):
            if start <= start_char < end:
                token_start_index = j
            if start < end_char <= end:
                token_end_index = j
                break

        start_positions.append(token_start_index)
        end_positions.append(token_end_index)

    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions

    # Remove the offset mappings, as they are no longer needed
    tokenized_examples.pop("offset_mapping")

    return tokenized_examples

# Apply the preprocessing function to the dataset
tokenized_squad = squad.map(add_token_positions, batched=True)


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

# import pretrind model

In [45]:

model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
num_train_epochs=2  # Lower the number of epochs


# TO INCREACE ACCURACY SET TRAINIG ARGUMRNT

In [50]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",  
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,  # Mixed precision training
    load_best_model_at_end=True  # Required for EarlyStoppingCallback
)


# split data in small size to get accuraccy

In [53]:
# Take a small fraction of the dataset (e.g., 10%)
small_train_dataset = tokenized_squad['train'].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_squad['validation'].shuffle(seed=42).select(range(30))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,  # Subset for faster training
    eval_dataset=small_eval_dataset,    # Smaller validation set
)

trainer.train()


  0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 5.021636486053467, 'eval_runtime': 14.1352, 'eval_samples_per_second': 2.122, 'eval_steps_per_second': 0.283, 'epoch': 1.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 4.923923969268799, 'eval_runtime': 14.8645, 'eval_samples_per_second': 2.018, 'eval_steps_per_second': 0.269, 'epoch': 2.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 4.89273738861084, 'eval_runtime': 14.4158, 'eval_samples_per_second': 2.081, 'eval_steps_per_second': 0.277, 'epoch': 3.0}
{'train_runtime': 509.9568, 'train_samples_per_second': 0.588, 'train_steps_per_second': 0.076, 'train_loss': 4.611846141326121, 'epoch': 3.0}


TrainOutput(global_step=39, training_loss=4.611846141326121, metrics={'train_runtime': 509.9568, 'train_samples_per_second': 0.588, 'train_steps_per_second': 0.076, 'total_flos': 39195930009600.0, 'train_loss': 4.611846141326121, 'epoch': 3.0})

    {'eval_loss': 5.021636486053467, 'eval_runtime': 14.1352, 'eval_samples_per_second': 2.122, 'eval_steps_per_second': 0.283, 'epoch': 1.0}
    {'eval_loss': 4.923923969268799, 'eval_runtime': 14.8645, 'eval_samples_per_second': 2.018, 'eval_steps_per_second': 0.269, 'epoch': 2.0}
    {'eval_loss': 4.89273738861084, 'eval_runtime': 14.4158, 'eval_samples_per_second': 2.081, 'eval_steps_per_second': 0.277, 'epoch': 3.0}
    {'train_runtime': 509.9568, 'train_samples_per_second': 0.588, 'train_steps_per_second': 0.076, 'train_loss': 4.611846141326121, 'epoch': 3.0}
    TrainOutput(global_step=39, training_loss=4.611846141326121, metrics={'train_runtime': 509.9568, 'train_samples_per_second': 0.588, 'train_steps_per_second': 0.076, 'total_flos': 39195930009600.0, 'train_loss': 4.611846141326121, 'epoch': 3.0})

# model evaluation to find score

In [54]:
# Evaluate the model performance on the validation set
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


  0%|          | 0/4 [00:00<?, ?it/s]

Evaluation results: {'eval_loss': 4.89273738861084, 'eval_runtime': 13.1403, 'eval_samples_per_second': 2.283, 'eval_steps_per_second': 0.304, 'epoch': 3.0}


    Evaluation results: {'eval_loss': 4.89273738861084, 'eval_runtime': 13.1403, 'eval_samples_per_second': 2.283, 'eval_steps_per_second': 0.304, 'epoch': 3.0}

# save dataset model to development 

In [ ]:
# Save the trained model
trainer.save_model("./trained_chatbot_model")

# Save the tokenizer if needed
tokenizer.save_pretrained("./trained_chatbot_model")
